# 자율주행 다중 물체 인식

In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 442.1/460.4 GB disk)


# data

In [2]:
import pandas as pd
import numpy as np
import os
import shutil
import cv2
import random
import json
import glob
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
origin_data_path = '/Users/kimhongseok/cv_79_projects/part2/23/origin_data/Training/바운딩박스/대전광역시'
images_path = '/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images'
labels_path = '/Users/kimhongseok/cv_79_projects/part2/23/origin_data/labels'

img_list = glob.glob(f'{origin_data_path}/원천/*/*.png')
label_list = glob.glob(f'{origin_data_path}/라벨/*/*.json')

'''
for img in img_list:
    name = img.split('/')[-1]
    shutil.copyfile(img, f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images/{name}')

for label in label_list:
    name = label.split('/')[-1]
    shutil.copyfile(label, f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/labels/{name}')
'''

"\nfor img in img_list:\n    name = img.split('/')[-1]\n    shutil.copyfile(img, f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images/{name}')\n\nfor label in label_list:\n    name = label.split('/')[-1]\n    shutil.copyfile(label, f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/labels/{name}')\n"

In [4]:
data_path = '/Users/kimhongseok/cv_79_projects/part2/23/data'
train_path = os.path.join(data_path, 'train')
valid_path = os.path.join(data_path, 'valid')
test_path = os.path.join(data_path, 'test')

for folder in [train_path, valid_path, test_path]:
    if not os.path.exists(folder):
        os.makedirs(folder)
        for s in ['images', 'labels']:
            s_folder = os.path.join(folder, s)
            if not os.path.exists(s_folder):
                os.makedirs(s_folder)

In [ ]:
'''
city_set = set()
gu_set = set()
weather_set = set()
gan_set = set()

for path in img_list:
    tmp = path.split('/')[-1]
    city = tmp.split('_')[1]
    gu = tmp.split('_')[3]
    weather = tmp.split('_')[4]
    gan = tmp.split('_')[5]
    city_set.add(city)
    gu_set.add(gu)
    weather_set.add(weather)
    gan_set.add(gan)

city = list(city_set)
gu = list(gu_set)
weather = list(weather_set)
gan = list(gan_set)

print(city, gu, weather, gan)
'''

In [ ]:
'''
file_list = os.listdir(f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images')
for file in file_list:
    name, ext = file.split('.')
    tmp = name.split('_')
    tmp[1] = str(city.index(tmp[1]))
    tmp[2] = str(city.index(tmp[2]))
    tmp[3] = str(gu.index(tmp[3]))
    tmp[4] = str(weather.index(tmp[4]))
    tmp[5] = str(gan.index(tmp[5]))
    new_name = '_'.join(tmp) + '.' + ext
    os.rename(f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images/{file}', f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/{i}/{new_name}')
'''

In [ ]:
'''
for i in ['images', 'labels']:
    file_list = os.listdir(f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/{i}')
    for file in file_list:
        name, ext = file.split('.')
        new_name = name.split('_')[-1]+'.'+ext
        os.rename(f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/{i}/{file}', f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/{i}/{new_name}')
'''

In [3]:
def xml_to_yolo_bbox(bbox, w, h):
    x_center = float(((bbox[2] + bbox[0])/2)/w)
    y_center = float(((bbox[3] + bbox[1])/2)/h)
    width = float((bbox[2] - bbox[0])/w)
    height = float((bbox[3] - bbox[1])/h)
    return [x_center, y_center, width, height]

In [ ]:
file_list = glob.glob('/Users/kimhongseok/cv_79_projects/part2/23/origin_data/labels/*.json')
print(len(file_list))

In [6]:
classes = ['car', 'pedestrian']
cars = ['일반차량', '목적차량(특장차)', '이륜차']

for file in file_list:
    result = list()
    save_name = file.replace('labels', 'txt_labels').replace('json', 'txt')
    with open(file, 'r') as f:
        data = json.load(f)
        h = int(data['camera']['resolution_height'])
        w = int(data['camera']['resolution_width'])
        for a in data['annotations']:
            if a['label'] in cars or a['label'] == '보행자':
                if a['label'] in cars:
                    index = 0
                else :
                    index = 1
                
                points = a['points']
                yolo_bbox = xml_to_yolo_bbox([int(points[0][0]), int(points[0][1]), int(points[1][0]), int(points[1][1])], w, h)
                yolo_string = ' '.join(list(str(x) for x in yolo_bbox))
                result.append(f'{index} {yolo_string}')

    if result:
        with open(save_name, 'w', encoding='utf-8') as f2:
            f2.write('\n'.join(result))

In [9]:
random.seed(2024)
file_list = glob.glob('/Users/kimhongseok/cv_79_projects/part2/23/origin_data/txt_labels/*.txt')
random.shuffle(file_list)
test_ratio = 0.1
num_file = len(file_list)

test_list = file_list[:int(num_file*test_ratio)]
valid_list = file_list[int(num_file*test_ratio):int(num_file*test_ratio)*2]
train_list = file_list[int(num_file*test_ratio)*2:]

for i in test_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{test_path}/labels/{label_name}')
    img_name = label_name.replace('txt', 'png')
    img_path = f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images/{img_name}'
    shutil.copyfile(img_path, f'{test_path}/images/{img_name}')

for i in valid_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{valid_path}/labels/{label_name}')
    img_name = label_name.replace('txt', 'png')
    img_path = f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images/{img_name}'
    shutil.copyfile(img_path, f'{valid_path}/images/{img_name}')

for i in train_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{train_path}/labels/{label_name}')
    img_name = label_name.replace('txt', 'png')
    img_path = f'/Users/kimhongseok/cv_79_projects/part2/23/origin_data/images/{img_name}'
    shutil.copyfile(img_path, f'{train_path}/images/{img_name}')

# config

In [ ]:
pjt_root = '/Users/kimhongseok/cv_79_projects/part2/23'
%cd /Users/kimhongseok/cv_79_projects/part2/23

In [14]:
import yaml
data = dict()

data['train'] = train_path
data['val'] = valid_path
data['test'] = test_path
data['nc'] = len(classes)
data['names'] = classes

with open(f'{pjt_root}/road_classification.yaml', 'w') as f:
    yaml.dump(data, f)

# train

In [5]:
import ultralytics
from ultralytics import YOLO

pjt_root = '/Users/kimhongseok/cv_79_projects/part2/23'
%cd /Users/kimhongseok/cv_79_projects/part2/23

/Users/kimhongseok/cv_79_projects/part2/23


In [6]:
model = YOLO('yolov8s.yaml')
results = model.train(data='road_classification.yaml', epochs=1, batch=16, patience=30, name='s')

New https://pypi.org/project/ultralytics/8.2.92 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=road_classification.yaml, epochs=1, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=s, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F

train: Scanning /Users/kimhongseok/cv_79_projects/part2/23/data/train/labels... 7228 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7228/7228 [00:06<00:00, 1054.34it/s]

train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0106286.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0106525.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0107112.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0107165.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0107227.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0107579.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0109973.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/train/images/0110154.png: 1 duplicate labels removed
train: WARNING ⚠️ /Users/kimhongseok/cv_79_proje

train: New cache created: /Users/kimhongseok/cv_79_projects/part2/23/data/train/labels.cache


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


...

val: Scanning /Users/kimhongseok/cv_79_projects/part2/23/data/valid/labels... 903 images, 0 backgrounds, 0 corrupt: 100%|██████████| 903/903 [00:00<00:00, 1115.16it/s]

val: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/valid/images/0107090.png: 1 duplicate labels removed
val: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/23/data/valid/images/0107108.png: 1 duplicate labels removed
val: New cache created: /Users/kimhongseok/cv_79_projects/part2/23/data/valid/labels.cache


Plotting labels to runs/detect/s/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/s
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G          0      126.5          0          0        640:   0%|          | 2/452 [00:30<1:53:07, 15.08s/it]


KeyboardInterrupt: 

# test

In [7]:
%cd /Users/kimhongseok/cv_79_projects/part2/23
pjt_root = '/Users/kimhongseok/cv_79_projects/part2/23'
result_folder = f'{pjt_root}/runs/detect'

/Users/kimhongseok/cv_79_projects/part2/23


In [ ]:
from ultralytics import YOLO

model = YOLO(f'{result_folder}/s/weights/best.pt')
metrics = model.val(split='test')

print('map50-95', metrics.box.map)
print('map50', metrics.box.map50)

# inference

In [ ]:
import torchvision.transforms as T